In [1]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torchvision.datasets import MNIST
from typing import Tuple

In [55]:
# shuffle_arr = [0,1,2,3,4,5,6,7,8,9]
shuffle_arr = [
    9, 8,
    7, 6,
    5, 4,
    3, 2,
    1, 0
    ]


In [56]:
class MyMNIST(MNIST):
    """
    Overrides the MNIST dataset to change the getitem function.
    """

    def __init__(self, root, train=True, transform=None,
                 target_transform=None, download=False) -> None:
        self.not_aug_transform = transforms.ToTensor()
        super(MyMNIST, self).__init__(root, train,
                                      transform, target_transform, download)

    def __getitem__(self, index: int) -> Tuple[Image.Image, int, Image.Image]:
        """
        Gets the requested element from the dataset.

        Args:
            index: index of the element to be returned

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.targets[index]
        target = shuffle_arr[target]



        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')
        original_img = self.not_aug_transform(img.copy())

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        if hasattr(self, 'logits'):
            return img, target, original_img, self.logits[index]

        return img, target, original_img

In [57]:
from torchvision import transforms
from PIL import Image

# Initialize the MyMNIST dataset
dataset = MyMNIST(root='path_to_data', train=True, transform=transforms.ToTensor(), download=True)

# Get an image and its target from the dataset at a specific index
img, target, original_img = dataset[0]  # Get the first image (index 0)

# Show the transformed image
img = transforms.ToPILImage()(img)  # Convert the tensor back to a PIL image for visualization
img.show()

# Show the original image (without transformations)
# original_img.show()

# You can also print the target class label
display(img)
print("Target class:", target)


Target class: 4


In [58]:
# mimic mammoth loader
target_array = []
img_array = []
for sample in dataset:
  img, target, original_img = sample
  target_array.append(target)
  img_array.append(img)

In [59]:
import numpy as np
target_array = np.array(target_array)
start_c = 0
end_c = 1
train_mask = np.logical_and(target_array >= start_c, target_array < end_c+1)


In [60]:
for i in range(5):
  start_c = 2*i
  end_c = 2*i + 1
  # print(f"start: {start_c} and end: {end_c}")
  train_mask = np.logical_and(target_array >= start_c, target_array < end_c+1)
  img_array = np.array(img_array)
  filtered_data = img_array[train_mask]
  filtered_class = target_array[train_mask]
  unique_elements = np.unique(filtered_class)
  print(unique_elements)

  for i in range(10):
    data = filtered_data[i][0]
    data = transforms.ToPILImage()(data)  # Convert the tensor back to a PIL image for visualization
    display(data)


[0 1]


[2 3]


[4 5]


[6 7]


[8 9]
